# Project Scoping

## 1. Project Goals

Some initial questions to answer that could be explored:

- What is the overall biodiversity across the parks?
- Are certain species more concentrated in specific parks?
- Are certain categories more endangered than others?
- Which parks have the most endangered species?
- Which species are most under threat?

## 2. Understanding the Dataset

There are two csv files: `species_info.csv` and `observations.csv`.
- `species_info.csv` columns: Animal category (eg. mammal), scientific name, common name, and conservation status.
- `observations.csv` columns: scientific name, park name, and number of observations of that animal over previous 7 days.

**Data Types**: all columns are string objects except for the number of observations which are int64.

**Size & Quality**:

- `species_info.csv`: 5,824 rows x 4 columns. Missing values: 5,633 (97%), all in the conservation status column.
- `observations.csv`: 23,296 rows x 3 columns. Missing values: 0

## 3. Key Metrics/Analysis Techniques

- Biodiversity Indexes to consider: Simpson’s Diversity Index, or Shannon Index to quantify biodiversity.
- Trend Analysis: Identify trends between parks.
- Visualization: Plan on using visualizations like bar charts, heatmaps, or maps to make the data easier to interpret.

## 4. Tools & Methods

Python Libraries: pandas, matplotlib, seaborn, for analysis and visualizations. Plotly for interactive map visualizations.

## 5. Deliverables

- Jupyter Notebook Report: Clear insights into biodiversity trends, backed by data.
- Linkedin Article: A summary of the findings and the importance of biodiversity conservation.
- Visualizations: Maps showing species distribution, charts for biodiversity indexes, etc.
- Conclusion: Summarize key findings.

---

# Boilerplate Structure for the Jupyter Notebook Report:

Project Title: Biodiversity in US National Parks
Your Name
Date
Introduction

Brief overview of the project.
Define the project goals (e.g., understanding biodiversity distribution, identifying trends, etc.).
Dataset Overview

Description of the dataset (source, size, variables).
Initial observations (any patterns or peculiarities?).
Data Cleaning & Preprocessing

Missing data handling, outliers, data transformation.
Tools and libraries used (e.g., pandas, NumPy).
Exploratory Data Analysis (EDA)

Visualizations of species diversity, park distributions, etc.
Any summary statistics or interesting patterns.
Species trends over time (if applicable).
Key Metrics & Calculations

Biodiversity indexes (e.g., Shannon Index, species richness).
Breakdown by park, species group, or region.
Insights & Findings

Highlight major trends or patterns found during analysis.
Discuss regional biodiversity differences, species at risk, etc.
Conservation Implications

Based on your findings, discuss potential conservation actions.
Any policy recommendations?
Conclusion

Summarize key takeaways.
Future work (e.g., further data collection, predictive modeling).
Appendix

Code snippets, additional graphs, or tables.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [48]:
# Load the data for each table
df_obs = pd.read_csv('observations.csv')
df_species = pd.read_csv('species_info.csv')

# Display the first few rows of each table, sorted by 'scientic_name' column
df_obs = df_obs.sort_values(by='scientific_name')
df_species = df_species.sort_values(by='scientific_name')
pd.set_option('display.width', 1000) # Set a large width to prevent wrapping
print('Observations:')
print(df_obs.head(10))
print('-' * 100)
print('Species Info:')
print(df_species.head(10))

Observations:
      scientific_name                            park_name  observations
8050    Abies bifolia            Yellowstone National Park           215
4968    Abies bifolia                  Bryce National Park           109
17990   Abies bifolia  Great Smoky Mountains National Park            72
20930   Abies bifolia               Yosemite National Park           136
4647   Abies concolor  Great Smoky Mountains National Park           101
15193  Abies concolor            Yellowstone National Park           241
23166  Abies concolor               Yosemite National Park           205
8170   Abies concolor                  Bryce National Park            83
2014    Abies fraseri                  Bryce National Park           109
11141   Abies fraseri            Yellowstone National Park           218
----------------------------------------------------------------------------------------------------
Species Info:
               category       scientific_name                       

To ease data analysis, it would be easier to consolidate the two datasets into one, using the scientific name as the primary key. This will allow for easier analysis and visualization of the data.

The common column is the column `scientific_name` in both datasets. It appears as though there might be four rows per species, one for each park, but this needs to be checked.

In [49]:
# Check the number of species (`scientific_name`) per park
species_per_park = df_obs.groupby('park_name')['scientific_name'].nunique()
print('Number of species per park:')
print(species_per_park)

Number of species per park:
park_name
Bryce National Park                    5541
Great Smoky Mountains National Park    5541
Yellowstone National Park              5541
Yosemite National Park                 5541
Name: scientific_name, dtype: int64


If there are 5541 distinct species, and 4 parks, there should be 5541 * 4 = 22164 rows, but there are 23,296 rows in the `observations.csv` file. This means there are an additional 1132 rows. This suggests that there are some species with more than one row for the same park.

In [50]:
# check which species have multiple observations for the same park:
species_obs_count = df_obs.groupby(['scientific_name', 'park_name'])['observations'].count()
species_obs_count = species_obs_count[species_obs_count > 1]
print('Species with multiple observations for the same park:')
print(species_obs_count)

Species with multiple observations for the same park:
scientific_name      park_name                          
Agrostis capillaris  Bryce National Park                    2
                     Great Smoky Mountains National Park    2
                     Yellowstone National Park              2
                     Yosemite National Park                 2
Agrostis gigantea    Bryce National Park                    2
                                                           ..
Vulpia octoflora     Yosemite National Park                 2
Zizia aptera         Bryce National Park                    2
                     Great Smoky Mountains National Park    2
                     Yellowstone National Park              2
                     Yosemite National Park                 2
Name: observations, Length: 1096, dtype: int64


In [51]:
# Add observations values for duplicates where scientific name and park name are the same.
df_obs_clean = df_obs.groupby(['scientific_name', 'park_name']).sum().reset_index()
print(df_obs_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22164 entries, 0 to 22163
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  22164 non-null  object
 1   park_name        22164 non-null  object
 2   observations     22164 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 519.6+ KB
None


In [15]:
# Merge the two tables on the 'scientific_name' column
df = pd.merge(df_obs, df_species, on='scientific_name')

# Sort the merged table by 'scientific_name' then 'park_name'
df = df.sort_values(by=['scientific_name', 'park_name'])

# Display the first few rows of the merged table
pd.set_option('display.width', 1000) # Set a large width to prevent wrapping
print(df.head(15))

            scientific_name                            park_name  observations           category                                       common_names conservation_status
5468          Abies bifolia                  Bryce National Park           109     Vascular Plant                          Rocky Mountain Alpine Fir                 NaN
19789         Abies bifolia  Great Smoky Mountains National Park            72     Vascular Plant                          Rocky Mountain Alpine Fir                 NaN
8854          Abies bifolia            Yellowstone National Park           215     Vascular Plant                          Rocky Mountain Alpine Fir                 NaN
23041         Abies bifolia               Yosemite National Park           136     Vascular Plant                          Rocky Mountain Alpine Fir                 NaN
8989         Abies concolor                  Bryce National Park            83     Vascular Plant  Balsam Fir, Colorado Fir, Concolor Fir, Silver...       

NB. the default merge in pandas is an inner merge, meaning only rows 

In [16]:
# check data types
print(df.dtypes)

scientific_name        object
park_name              object
observations            int64
category               object
common_names           object
conservation_status    object
dtype: object


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25632 entries, 5468 to 17885
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   scientific_name      25632 non-null  object
 1   park_name            25632 non-null  object
 2   observations         25632 non-null  int64 
 3   category             25632 non-null  object
 4   common_names         25632 non-null  object
 5   conservation_status  880 non-null    object
dtypes: int64(1), object(5)
memory usage: 1.4+ MB


In [26]:
# check for missing values
print(df.isnull().sum())

scientific_name            0
park_name                  0
observations               0
category                   0
common_names               0
conservation_status    24752
dtype: int64


In [27]:
# print unique values of conservation_status
print(df.conservation_status.unique())

[nan 'Species of Concern' 'Threatened' 'Endangered' 'In Recovery']
